In [35]:
import time
import torch

device = "mps"
msize = 5000
tloop = 100

torch.manual_seed(1234)
TENSOR_A_CPU = torch.rand(msize, msize)
TENSOR_B_CPU = torch.rand(msize, msize)

torch.manual_seed(1234)
TENSOR_A_MPS = torch.rand(msize, msize).to(device)
TENSOR_B_MPS = torch.rand(msize, msize).to(device)

In [36]:
start_time = time.time()

for _ in range(tloop):
    TENSOR_A_CPU @ TENSOR_B_CPU

print(f"CPU : --- {time.time() - start_time} seconds ---")

CPU : --- 11.547360897064209 seconds ---


In [37]:
start_time = time.time()

for _ in range(tloop):
    TENSOR_A_MPS @ TENSOR_B_MPS

print(f"MPS : --- {time.time() - start_time} seconds ---")

MPS : --- 0.016175031661987305 seconds ---


In [38]:
TENSOR_A_MPS.shape

torch.Size([5000, 5000])